In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("Petitions.csv")

data

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство
...,...,...,...
59884,3128111,прошу закрасить граффити,Благоустройство
59885,3276713,Прошу вас отремонтировать пешеходную дорожку,Благоустройство
59886,3274663,Необходимо демонтировать незаконную рекламную ...,Незаконная информационная и (или) рекламная ко...
59887,3359308,Очень гремит на ветру металлическая часть окна...,Кровля


In [ ]:
data = data.drop(columns="id")

data

,public_petition_text,reason_category
0,снег на дороге,Благоустройство
1,очистить кабельный киоск от рекламы,Благоустройство
2,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,Граффити,Благоустройство
...,...,...
59884,прошу закрасить граффити,Благоустройство
59885,Прошу вас отремонтировать пешеходную дорожку,Благоустройство
59886,Необходимо демонтировать незаконную рекламную ...,Незаконная информационная и (или) рекламная ко...
59887,Очень гремит на ветру металлическая часть окна...,Кровля


# Предобработка данных

In [ ]:
!pip install pymorphy3
import re

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('russian')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.9 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocessing(text):
    text_without_special_characters = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', text)

    words = word_tokenize(text_without_special_characters)

    normal_forms = [morph.normal_forms(word)[0] for word in words]

    delete_stopwords = [word for word in normal_forms if word not in stopwords]

    return normal_forms

preprocessing_data = [preprocessing(x) for x in data['public_petition_text'].to_numpy()[0:1000]]

preprocessing_data

[['снег', 'на', 'дорога'],
 ['очистить', 'кабельный', 'киоск', 'от', 'реклама'],
 ['просить',
  'убрать',
  'всё',
  'дерево',
  'и',
  'кустарник',
  'который',
  'выйти',
  'за',
  'предел',
  'газон',
  'на',
  'пешеходный',
  'зона',
  'начинать',
  'от',
  'подъезд',
  '13',
  'до',
  'подъезд',
  '15',
  'фасад',
  'дом',
  'с',
  'сторона',
  'ул',
  'наличный'],
 ['неудовлетворительный',
  'состояние',
  'парадный',
  'надпись',
  'на',
  'дверь',
  '2',
  'этаж'],
 ['граффити'],
 ['необходимо',
  'проверить',
  'законность',
  'установка',
  'вывеска',
  'на',
  'фасад',
  'мкд',
  'по',
  'адрес',
  'проспект',
  'непокора',
  'нный',
  '74',
  'в',
  'случай',
  'если',
  'вывеска',
  'установить',
  'незаконно',
  'быть',
  'необходимо',
  'демонтировать'],
 ['уборка',
  'не',
  'производиться',
  'на',
  'лестница',
  'очень',
  'грязно',
  'на',
  'весь',
  'этаж',
  'вплоть',
  'до',
  '5',
  'го',
  'звонок',
  'в',
  'жкс',
  '2',
  '5',
  '04',
  '2021',
  'не',
  'да

In [ ]:
import torch
words = set(word for x in preprocessing_data for word in x)

dict_ = {word:indx for indx, word in enumerate(words)}
dict_2 = {indx:word for indx, word in enumerate(words)}

context_embedding_matrix = torch.zeros(len(dict_), len(dict_))

for word in words:
    for sentence in preprocessing_data:
        if word in sentence:
            for word_sentence in sentence:
                context_embedding_matrix[dict_[word]][dict_[word_sentence]] += 1

for i in range(len(words)):
    context_embedding_matrix[i][i] = 0

context_embedding_matrix



tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=25)

pca_data = pca.fit_transform(context_embedding_matrix)

# CBOW

In [ ]:
import sys
sys.path.append('CBOW.py')

In [ ]:
data_context = []

context_size = 2

train_data = []
test_data = []

j = 0
for sentence in preprocessing_data:
    if len(sentence) <= context_size * 2:
        continue
    for i in range(2, len(sentence) - 2):
        data_context.append([[sentence[i - 2], sentence[i - 1], sentence[i + 1], sentence[i + 2]], sentence[i]])
        if (j % 10 == 0):
            test_data.append([[sentence[i - 2], sentence[i - 1], sentence[i + 1], sentence[i + 2]], sentence[i]])
        else:
            train_data.append([[sentence[i - 2], sentence[i - 1], sentence[i + 1], sentence[i + 2]], sentence[i]])
        j += 1

# train_data = random.sample(data_context[:-100], len(data_context) // 3)
# test_data = data_context[-100:]
data_context

[[['очистить', 'кабельный', 'от', 'реклама'], 'киоск'],
 [['просить', 'убрать', 'дерево', 'и'], 'всё'],
 [['убрать', 'всё', 'и', 'кустарник'], 'дерево'],
 [['всё', 'дерево', 'кустарник', 'который'], 'и'],
 [['дерево', 'и', 'который', 'выйти'], 'кустарник'],
 [['и', 'кустарник', 'выйти', 'за'], 'который'],
 [['кустарник', 'который', 'за', 'предел'], 'выйти'],
 [['который', 'выйти', 'предел', 'газон'], 'за'],
 [['выйти', 'за', 'газон', 'на'], 'предел'],
 [['за', 'предел', 'на', 'пешеходный'], 'газон'],
 [['предел', 'газон', 'пешеходный', 'зона'], 'на'],
 [['газон', 'на', 'зона', 'начинать'], 'пешеходный'],
 [['на', 'пешеходный', 'начинать', 'от'], 'зона'],
 [['пешеходный', 'зона', 'от', 'подъезд'], 'начинать'],
 [['зона', 'начинать', 'подъезд', '13'], 'от'],
 [['начинать', 'от', '13', 'до'], 'подъезд'],
 [['от', 'подъезд', 'до', 'подъезд'], '13'],
 [['подъезд', '13', 'подъезд', '15'], 'до'],
 [['13', 'до', '15', 'фасад'], 'подъезд'],
 [['до', 'подъезд', 'фасад', 'дом'], '15'],
 [['подъез

In [ ]:
def make_context_vector(context, dict_):
    return torch.tensor([dict_[word] for word in context], dtype=torch.long)

In [ ]:
import CBOW
import torch.nn as nn

vocab_size = len(dict_)
embedding_size = 100

model = CBOW.CBOW(vocab_size, embedding_size)

# определяем функцию потерь
loss = nn.CrossEntropyLoss()
# настраиваем оптимизатор и передаем туда параметры модели
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 5

for epoch in range(epochs):
    # за одну эпоху смотрим все данные {батчи} (по batch_size элементов)
    for context, target in train_data[:len(train_data) // 2]:
        context_vector = make_context_vector(context, dict_)
        y_vector = torch.tensor([dict_[target]], dtype=torch.long)
        # делаем прямое распространение (получаем предсказание)
        outputs = model(context_vector)
        # вычисляем значение функции потерь
        loss_value = loss(outputs, y_vector)
        # делаем backward - вычисляются значения .grad у слоев модели
        loss_value.backward()
        # делаем шаг градиентного спуска с заданным у оптимизатора learning_rate
        optimizer.step()
        # зануляем .grad у слоев модели - для нового батча будем акумулировать новый .grad
        optimizer.zero_grad()


    # в конце эпохи выводим значение функции потерь для последнего рассмотренного батча
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 11.156208038330078
Эпоха 2, Значение функции потерь: 2.070919990539551
Эпоха 3, Значение функции потерь: 0.0013679441763088107
Эпоха 4, Значение функции потерь: 0.0048653543926775455
Эпоха 5, Значение функции потерь: 0.0004866131057497114


In [ ]:
t = 0
for x in test_data:
    if (dict_2[torch.argmax(model(make_context_vector(x[0], dict_))).item()] == x[1]):
        t += 1

t, len(test_data)

(123, 936)

In [ ]:


with open('context_matrix_PCA.tsv', 'w') as file:
    for row in context_embedding_matrix.numpy():
        file.write('\t'.join(str(x) for x in row) + '\n')

In [ ]:
with open('context_matrix_PCA_2.tsv', 'w') as file2:
    file2.write(word + '\n' for word in list(words))

TypeError: ignored

In [ ]:
matrix = [word + '\n' for word in list(words)]
matrix
text = ""
for i in matrix :
    text += i

In [ ]:
with open('context_matrix_PCA_3.tsv', 'w') as file3:
    file3.write('\n'.join(text) + '\n')